# Project Prometheus v0.16: The Parallel Mind

This notebook contains the full, executable implementation of the Project Prometheus v0.16 Proof of Concept. It is designed to be run in a Google Colab environment with a GPU runtime.

## 1. Setup

In [ ]:
!pip install google-generativeai pydantic pypdf pyvis causallearn

## 2. Imports and API Key

In [ ]:
import os
import logging
import json
import time
from prometheus import *

# --- API Key Setup ---
API_KEY = "YOUR_API_KEY" # <--- IMPORTANT: PASTE YOUR API KEY HERE
os.environ["GOOGLE_API_KEY"] = API_KEY

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## 3. Verification Tests

### 3.1 Test: Parallel Experimentation

In [ ]:
from prometheus.experiment_orchestrator import ExperimentOrchestrator

planner = PlannerAgent()
hypotheses = planner.generate_hypotheses("Synthesize molecule D")

orchestrator = ExperimentOrchestrator()
start_time = time.time()
results = orchestrator.run_parallel_experiments(hypotheses)
end_time = time.time()

logging.info(f"Total time for parallel execution: {end_time - start_time:.2f}s")
assert len(results) == len(hypotheses)

### 3.2 Test: Causal Inference

In [ ]:
from prometheus.results_synthesizer import ResultsSynthesizer
from prometheus.tools import CausalGraphTool

# Simulate a new rule for the ToyChemistrySim where mixing A and C produces D
def new_mix(self, c1, c2):
    if {c1, c2} == {'A', 'C'}:
        self.state['D'] = 100
    else:
        self.state['D'] = 0
    return self.state
ToyChemistrySim.mix = new_mix

results = orchestrator.run_parallel_experiments(hypotheses)
causal_tool = CausalGraphTool()
synthesizer = ResultsSynthesizer(causal_tool)
causal_graph = synthesizer.analyze_results(results)

knowledge_agent = KnowledgeAgent(api_key=API_KEY, performance_logger=None, pdf_tool=None)
knowledge_agent.update_causal_rules(causal_graph)
assert 'mix_A_C' in knowledge_agent.get_causal_rules()[0]

## 4. Demonstration: The Automated Scientist

In [ ]:
from prometheus.brain_map import BrainMap
from prometheus.planner import PlannerAgent
from prometheus.knowledge_agent import KnowledgeAgent
from prometheus.mcs import MCSSupervisor
from IPython.display import display, HTML

brain_map = BrainMap()
planner = PlannerAgent()
knowledge_agent = KnowledgeAgent(api_key=API_KEY, performance_logger=None, pdf_tool=None)
supervisor = MCSSupervisor(planner, None, knowledge_agent, brain_map)

goal = "Discover the reaction pathway to synthesize molecule D."
supervisor.run_discovery_cycle(goal)

display(HTML('brain_map.html'))